In [3]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

In [33]:
# open the data file
with h5py.File('../CodePass2/runs/stas_comments_10.hdf5', 'r') as f:
    # read the data
    x = f['x'][:]
 

print(x.shape)


step = x[1]


# find voronoi diagram
from scipy.spatial import Delaunay

tri = Delaunay(step)



(1001, 5000, 3)


In [34]:
indsx, all_neighbors = tri.vertex_neighbor_vertices

neighbors = []
for i in range(len(indsx)-1):
    neighbors.append(all_neighbors[indsx[i]:indsx[i+1]])



In [32]:
neighbors

[array([  12, 4973, 4525, 2997, 3814, 2911, 1162], dtype=int32),
 array([3430, 3659, 2566, 1534, 3771, 3375, 2902, 4168, 2822, 1944, 4005],
       dtype=int32),
 array([4906, 2092, 1093, 2727, 3086, 3084, 2366,  822, 3246, 4870, 2367,
        2246], dtype=int32),
 array([4916, 2753, 3118, 2385, 1525,  991, 1513, 1070], dtype=int32),
 array([4427, 2184, 1106,  331, 4307, 4444, 1770, 2918, 2996, 3843, 2891,
        3575,  952,  225,  566, 3485, 4729, 3159,  266,  603, 3716,  902,
        4881, 3869,  268, 3308, 3043,  171], dtype=int32),
 array([4793, 1246, 1415, 4557, 4951, 1052, 3266, 4698, 2019, 3280, 4335,
        2651, 4449, 1085, 4272], dtype=int32),
 array([3949, 2405, 2395, 3817, 1725,  258, 2423, 2041, 1294,  869,  891,
        3761,  793], dtype=int32),
 array([4016,  972, 1018, 1672, 2000, 3880, 3427, 3611, 2186, 2063, 2147,
        4071], dtype=int32),
 array([ 264, 3675, 3702, 1509, 1826, 3301, 1699,  616, 3693, 2572, 1799],
       dtype=int32),
 array([ 764, 3404, 2028, 416

In [18]:
dir(vor)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_points',
 '_points',
 '_qhull',
 '_ridge_dict',
 '_update',
 'add_points',
 'close',
 'furthest_site',
 'max_bound',
 'min_bound',
 'ndim',
 'npoints',
 'point_region',
 'points',
 'regions',
 'ridge_dict',
 'ridge_points',
 'ridge_vertices',
 'vertices']